<a href="https://colab.research.google.com/github/Vlad12344/Inverted-Pendulum-Project/blob/master/Kaggle_RAM_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import os
import re
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.preprocessing import image
from PIL import Image, ImageDraw, ImageOps
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.optimizers import Adadelta
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import regularizers
from keras.applications import MobileNet, VGG16
from keras.models import model_from_json
from keras.layers import BatchNormalization

from google.colab import drive
drive.mount('/content/drive/')

# seed для повторяемости результатов
np.random.seed(42)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
size = 32
#PATH = '/content/drive/My Drive/colab/dataset_32x32/random_dat'
PATH = '/content/drive/My Drive/colab/dataset_50'

In [0]:
X_train = np.load(os.path.join(PATH, 'X_train.npy'))
Y_train = np.load(os.path.join(PATH, 'Y_train.npy'))
x_test = np.load(os.path.join(PATH, 'x_test.npy'))
y_test = np.load(os.path.join(PATH, 'y_test.npy'))

x_test = x_test.astype('float16')
X_train = X_train.astype('float16')

X_train /= 255
x_test /= 255

y_test = np_utils.to_categorical(y_test)
Y_train = np_utils.to_categorical(Y_train)

#y_test = np.delete(y_test, np.s_[0:0], 1)
#Y_train = np.delete(Y_train, np.s_[0:0], 1)

In [0]:
model = Sequential()

model.add(Convolution2D(filters = 64, kernel_size = (3, 3), padding = 'same',
                 input_shape = (size,size,1), activation = 'relu', 
                 data_format = "channels_last"))
#model.add(BatchNormalization())

model.add(Convolution2D(filters = 64, kernel_size = (3, 3), padding = 'valid',
                 activation = 'relu', data_format = "channels_last"))
#model.add(BatchNormalization())


model.add(MaxPooling2D(pool_size = (2,2), 
                       data_format = "channels_last"))
model.add(Dropout(0.25))

model.add(Convolution2D(filters = 128, kernel_size = (3, 3), padding = 'valid',
                       activation = 'relu', data_format = "channels_last"))
#model.add(BatchNormalization())
          
model.add(Convolution2D(filters = 128, kernel_size = (3, 3), padding = 'valid',
                       activation = 'relu', data_format = "channels_last"))
#model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size = (2,2), 
                       data_format = "channels_last"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(120, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(200, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation = 'softmax'))

In [0]:
model.summary()

In [0]:
optim = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer=optim, 
              metrics=['accuracy'])

In [52]:
for i in range(1):
  
  history = model.fit(X_train, Y_train, epochs=1, validation_split=0.15, shuffle=True, batch_size=500)

  model.save_weights(os.path.join(PATH, 'quick_doodle.h5'))

  model_json = model.to_json()
# Записываем модель в файл
  json_file = open(os.path.join(PATH, "quick_doodle.json"), 'w')
  json_file.write(model_json)
  json_file.close()

Train on 255000 samples, validate on 45000 samples
Epoch 1/1
255000/255000 [==============================] - 80s 312us/step - loss: 3.0840 - acc: 0.1953 - val_loss: 1.9355 - val_acc: 0.4878


In [55]:
scores = model.predict(x_test, batch_size = 128, verbose=1)
print(np.max(scores[0,:]))

50000/50000 [==============================] - 6s 111us/step
0.46782935


model_json = model.to_json()

json_file = open('/media/vlados/FreeSpace/Kaggle/dataset/quick_doodle.h5', "w")
json_file.write(model_json)
json_file.close()

In [0]:
model.save_weights(os.path.join(PATH, 'quick_doodle.h5'))

In [0]:
#json_file = open("/content/drive/My Drive/colab/dataset_32x32/quick_doodle_M.json", "r")
#loaded_model_json = json_file.read()
#json_file.close()
# Создаем модель на основе загруженных данных
#loaded_model = model_from_json(loaded_model_json)
# Загружаем веса в модель
loaded_model.load_weights('/content/drive/My Drive/colab/dataset_32x32/quick_doodle.h5')

In [0]:
loaded_model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

loaded_model.fit(X_train, Y_train, epochs=5, validation_split=0.2, shuffle=True, batch_size=128)

scores = loaded_model.evaluate(x_test, y_test, verbose=0)
print("Точность модели на тестовых данных: %.2f%%" % (scores[1]*100))

In [0]:
model_json = model.to_json()
# Записываем модель в файл
json_file = open("/content/drive/My Drive/colab/dataset_32x32/random_dat/quick_doodle.json", "w")
json_file.write(model_json)
json_file.close()

In [0]:
#classes = np.load(os.path.join(PATH,'classes.npy'))
optim = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)

model = MobileNet(input_shape=(size, size, 1), alpha=1, weights=None, classes=50)
model.compile(optimizer=optim, loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
for i in range(10):
  
  history = model.fit(X_train, Y_train, epochs=1, validation_split=0.15, shuffle=True, batch_size=128) 
  model.save_weights(os.path.join(PATH, 'quick_doodle_M.h5'))

  model_json = model.to_json()
# Записываем модель в файл
  json_file = open(os.path.join(PATH, "quick_doodle_M.json"), 'w')
  json_file.write(model_json)
  json_file.close()

In [33]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Точность модели на тестовых данных: %.2f%%" % (scores[1]*100))

Точность модели на тестовых данных: 1.24%


In [0]:
x_test = np.load('/content/drive/My Drive/colab/dataset_32x32/random_dat/x__test.npy')

In [0]:
import re

Input_dir = '/content/drive/My Drive/colab/dataset/' #csv files path
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

def top3(predictions):
    
    classes = np.load('/content/drive/My Drive/colab/dataset_32x32/random_dat/classes.npy')
    top_3 =  np.argsort(-predictions)[:, 0:3]
    top_3_name = np.empty(top_3.shape, dtype=object)
    
    for row in range(len(top_3)):
        for element in range(3):
            top_3_name[row,element] = re.sub(' ', '_', classes[top_3[row,element]]) 
    
    return(top_3_name)
  
top = top3(test_predictions)

preds_df = pd.DataFrame({'first': top[:,0], 'second': top[:,1], 'third': top[:,2]})
preds_df['words'] = preds_df['first'] + " " + preds_df['second'] + " " + preds_df['third']

sub = pd.read_csv(Input_dir + 'sample_submission.csv', index_col=['key_id'])
sub['word'] = preds_df.words.values
sub.to_csv('/content/drive/My Drive/colab/dataset_32x32/1class_per_label_proto_M.csv')
sub.head()

In [0]:
def model_import(submit_file):
  json_file = open(os.path.join(PATH, '50_1', 'quick_doodle.json'), "r")
  loaded_model_json = json_file.read()
  json_file.close()
# Создаем модель на основе загруженных данных
  loaded_model_1 = model_from_json(loaded_model_json)
# Загружаем веса в модель
  loaded_model_1.load_weights(os.path.join(PATH, '50_1', 'quick_doodle.h5'))
  
  json_file = open(os.path.join(PATH, '50_2', 'quick_doodle.json'), "r")
  loaded_model_json = json_file.read()
  json_file.close()
# Создаем модель на основе загруженных данных
  loaded_model_2 = model_from_json(loaded_model_json)
# Загружаем веса в модель
  loaded_model_2.load_weights(os.path.join(PATH, '50_2', 'quick_doodle.h5'))
  
  json_file = open(os.path.join(PATH, '50_3', 'quick_doodle.json'), "r")
  loaded_model_json = json_file.read()
  json_file.close()
# Создаем модель на основе загруженных данных
  loaded_model_3 = model_from_json(loaded_model_json)
# Загружаем веса в модель
  loaded_model_3.load_weights(os.path.join(PATH, '50_3', 'quick_doodle.h5'))
  
  json_file = open(os.path.join(PATH, '50_4', 'quick_doodle.json'), "r")
  loaded_model_json = json_file.read()
  json_file.close()
# Создаем модель на основе загруженных данных
  loaded_model_4 = model_from_json(loaded_model_json)
# Загружаем веса в модель
  loaded_model_4.load_weights(os.path.join(PATH, '50_4', 'quick_doodle.h5'))
  
  json_file = open(os.path.join(PATH, '50_5', 'quick_doodle.json'), "r")
  loaded_model_json = json_file.read()
  json_file.close()
# Создаем модель на основе загруженных данных
  loaded_model_5 = model_from_json(loaded_model_json)
# Загружаем веса в модель
  loaded_model_5.load_weights(os.path.join(PATH, '50_5', 'quick_doodle.h5'))
  
  json_file = open(os.path.join(PATH, '50_6', 'quick_doodle.json'), "r")
  loaded_model_json = json_file.read()
  json_file.close()
# Создаем модель на основе загруженных данных
  loaded_model_6 = model_from_json(loaded_model_json)
# Загружаем веса в модель
  loaded_model_6.load_weights(os.path.join(PATH, '50_6', 'quick_doodle.h5'))
  
  json_file = open(os.path.join(PATH, '50_7', 'quick_doodle.json'), "r")
  loaded_model_json = json_file.read()
  json_file.close()
# Создаем модель на основе загруженных данных
  loaded_model_7 = model_from_json(loaded_model_json)
# Загружаем веса в модель
  loaded_model_7.load_weights(os.path.join(PATH, '50_7', 'quick_doodle.h5'))
  
  pred_1 = loaded_model_1.predict(submit_file, batch_size=128, verbose=1)
  pred_2 = loaded_model_2.predict(submit_file, batch_size=128, verbose=1)
  pred_3 = loaded_model_3.predict(submit_file, batch_size=128, verbose=1)
  pred_4 = loaded_model_4.predict(submit_file, batch_size=128, verbose=1)
  pred_5 = loaded_model_5.predict(submit_file, batch_size=128, verbose=1)
  pred_6 = loaded_model_6.predict(submit_file, batch_size=128, verbose=1)
  pred_7 = loaded_model_7.predict(submit_file, batch_size=128, verbose=1)
  
  total_pred = np.concatenate((pred_1, pred_2, pred_3, 
                              pred_4,pred_5, pred_6, pred_7), axis=1, )
  return(total_pred)

In [53]:
x_test = np.load(os.path.join(PATH, 'x__test.npy'))

model = MobileNet(input_shape=(size, size, 1), alpha=1, weights=os.path.join(PATH, 'quick_doodle_M.h5'), classes=340)
predict_mobile = model.predict(x_test, batch_size=128, verbose=1)

json_file = open(os.path.join(PATH, 'quick_doodle.json'), "r")
loaded_model_json = json_file.read()
json_file.close()

# Создаем модель на основе загруженных данных
loaded_model = model_from_json(loaded_model_json)

# Загружаем веса в модель
loaded_model.load_weights(os.path.join(PATH, 'quick_doodle.h5'))

predict_standart = loaded_model.predict(x_test, batch_size=128, verbose=1)

predictions = (predict_standart + predict_mobile) / 2

def top3(predictions):
    
    classes = np.load(os.path.join(PATH, 'classes.npy'))
    top_3 =  np.argsort(-predictions)[:, 0:3]
    top_3_name = np.empty(top_3.shape, dtype=object)
    
    for row in range(len(top_3)):
        for element in range(3):
            top_3_name[row,element] = re.sub(' ', '_', classes[top_3[row,element]]) 
    
    return(top_3_name)
  
top = top3(predictions)

preds_df = pd.DataFrame({'first': top[:,0], 'second': top[:,1], 'third': top[:,2]})
preds_df['words'] = preds_df['first'] + " " + preds_df['second'] + " " + preds_df['third']

sub = pd.read_csv('/content/drive/My Drive/colab/dataset/' + 'sample_submission.csv', index_col=['key_id'])
sub['word'] = preds_df.words.values
sub.to_csv('/content/drive/My Drive/colab/dataset_32x32/1class_per_label_proto_Mst.csv')
sub.head()

FileNotFoundError: ignored

In [82]:
def top3(predictions):
    
    classes = np.load(os.path.join(PATH, 'classes.npy'))
    top_3 =  np.argsort(-predictions)[:, 0:3]
    top_3_name = np.empty(top_3.shape, dtype=object)
    
    for row in range(len(top_3)):
        for element in range(3):
            top_3_name[row,element] = re.sub(' ', '_', classes[top_3[row,element]]) 
    
    return(top_3_name)

x_test = np.load(os.path.join(PATH, 'x__test.npy'))

predictions = model_import(x_test)

top = top3(predictions)

preds_df = pd.DataFrame({'first': top[:,0], 'second': top[:,1], 'third': top[:,2]})
preds_df['words'] = preds_df['first'] + " " + preds_df['second'] + " " + preds_df['third']

sub = pd.read_csv('/content/drive/My Drive/colab/dataset/' + 'sample_submission.csv', index_col=['key_id'])
sub['word'] = preds_df.words.values
sub.to_csv('/content/drive/My Drive/colab/dataset_32x32/1class_per_label_proto_Mst.csv')
sub.head()

112199/112199 [==============================] - 17s 154us/step


,word
key_id,
9000003627287624,radio stereo train
9000010688666847,hockey_puck sandwich steak
9000023642890129,The_Great_Wall_of_China castle squiggle
9000038588854897,mountain triangle pliers
9000052667981386,campfire fireplace kangaroo


In [96]:
(predictions[0,150:200])

array([1.17698126e-01, 1.03414319e-02, 3.47257010e-03, 9.38992202e-03,
       1.05952518e-03, 5.21875045e-04, 1.05166237e-03, 4.47650766e-03,
       1.22260640e-03, 2.09903880e-03, 1.21325754e-01, 1.18842057e-03,
       2.36647879e-03, 1.30987708e-02, 4.90916878e-01, 5.60439366e-04,
       2.37089372e-03, 9.04475222e-04, 2.46729632e-03, 7.09617627e-04,
       7.31880544e-04, 1.37150288e-03, 6.79366349e-04, 2.37184041e-03,
       9.51877664e-05, 4.99094371e-03, 3.27919573e-02, 8.15031026e-03,
       2.21034344e-02, 2.80362163e-02, 6.35030621e-04, 1.07308291e-03,
       3.56790088e-02, 2.94828322e-04, 2.31127240e-04, 1.19931647e-03,
       2.21713725e-02, 1.98587004e-04, 1.84167305e-03, 4.55079228e-03,
       8.74905661e-03, 5.70928957e-03, 1.64937240e-03, 1.90193078e-03,
       3.55405034e-03, 5.11326408e-03, 7.24898302e-04, 1.39797824e-02,
       4.43272991e-04, 1.73526630e-03], dtype=float32)

In [86]:
x_test = np.load('/content/drive/My Drive/colab/dataset_50/.npy')
json_file = open('/content/drive/My Drive/colab/dataset_50/50_3/quick_doodle.json', "r")
loaded_model_json = json_file.read()
json_file.close()
# Создаем модель на основе загруженных данных
loaded_model_2 = model_from_json(loaded_model_json)
# Загружаем веса в модель
loaded_model_2.load_weights('/content/drive/My Drive/colab/dataset_50/50_3/quick_doodle.h5')
  
pred = loaded_model_2.predict(x_test, batch_size=128, verbose=1)
  
pred[:,:]

50000/50000 [==============================] - 10s 192us/step


array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 1.6582736e-31, 0.0000000e+00, ..., 3.4264788e-19,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [87]:
print(x_test)

[[[[  0]
   [  0]
   [  0]
   ...
   [ 92]
   [192]
   [208]]

  [[  0]
   [  0]
   [  0]
   ...
   [119]
   [  8]
   [192]]

  [[  0]
   [  0]
   [  0]
   ...
   [  0]
   [ 28]
   [187]]

  ...

  [[  0]
   [  0]
   [  0]
   ...
   [  0]
   [  0]
   [  0]]

  [[  0]
   [  0]
   [  0]
   ...
   [  0]
   [  0]
   [  0]]

  [[  0]
   [  0]
   [  0]
   ...
   [  0]
   [  0]
   [  0]]]


 [[[  0]
   [  0]
   [  0]
   ...
   [  0]
   [  0]
   [  0]]

  [[  0]
   [  0]
   [  0]
   ...
   [  0]
   [  0]
   [  0]]

  [[  0]
   [  0]
   [  0]
   ...
   [  0]
   [  0]
   [  0]]

  ...

  [[177]
   [  0]
   [  0]
   ...
   [  0]
   [  0]
   [  0]]

  [[199]
   [  1]
   [  0]
   ...
   [  0]
   [  0]
   [  0]]

  [[143]
   [121]
   [ 96]
   ...
   [  0]
   [  0]
   [  0]]]


 [[[115]
   [160]
   [128]
   ...
   [122]
   [  7]
   [  0]]

  [[127]
   [ 65]
   [  0]
   ...
   [146]
   [ 66]
   [  0]]

  [[115]
   [ 77]
   [  0]
   ...
   [192]
   [ 20]
   [  0]]

  ...

  [[177]
   [ 66]
   [ 10]
   